## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description,Country,Date
0,0,Jamestown,42.0970,-79.2353,23.09,69,100,5.01,snow,US,2022-02-04 03:07:54
1,1,Bambous Virieux,-20.3428,57.7575,80.22,88,99,21.50,light rain,MU,2022-02-04 03:13:39
2,2,Espinosa,-14.9261,-42.8192,70.30,87,43,3.58,scattered clouds,BR,2022-02-04 03:13:40
3,3,Cherskiy,68.7500,161.3000,-28.86,99,16,3.36,few clouds,RU,2022-02-04 03:10:08
4,4,Udachnyy,66.4167,112.4000,-22.68,99,98,2.51,overcast clouds,RU,2022-02-04 03:12:25


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_city_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description,Country,Date
2,2,Espinosa,-14.9261,-42.8192,70.30,87,43,3.58,scattered clouds,BR,2022-02-04 03:13:40
10,10,Coari,-4.0850,-63.1414,73.98,98,100,1.52,overcast clouds,BR,2022-02-04 03:13:42
18,18,Luderitz,-26.6481,15.1594,68.16,70,1,15.68,clear sky,NaN,2022-02-04 03:10:49
23,23,Sao Filipe,14.8961,-24.4956,72.52,77,0,8.75,clear sky,CV,2022-02-04 03:12:53
28,28,Rio Grande,-32.0350,-52.0986,70.95,86,100,10.00,light rain,BR,2022-02-04 03:10:38
33,33,Senanga,-16.1167,23.2667,65.39,95,97,2.48,light rain,ZM,2022-02-04 03:13:49
37,37,Tocopilla,-22.0920,-70.1979,68.99,75,14,8.21,few clouds,CL,2022-02-04 03:13:50
44,44,Victoria,22.2855,114.1577,61.18,77,67,1.99,broken clouds,HK,2022-02-04 03:10:36
47,47,East London,-33.0153,27.9116,69.28,80,100,15.37,light rain,ZA,2022-02-04 03:12:30
52,52,Cabo San Lucas,22.8909,-109.9124,67.51,58,20,8.05,few clouds,MX,2022-02-04 03:14:32


In [5]:
# 4a. Determine if there are any empty rows.
preferred_city_df.count()

City_ID             176
City                176
Lat                 176
Lng                 176
Max Temp            176
Humidity            176
Cloudiness          176
Wind Speed          176
City Description    176
Country             169
Date                176
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_city_df.dropna()
clean_df.count()

City_ID             169
City                169
Lat                 169
Lng                 169
Max Temp            169
Humidity            169
Cloudiness          169
Wind Speed          169
City Description    169
Country             169
Date                169
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "City Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,City Description,Lat,Lng,Hotel Name
2,Espinosa,BR,70.30,scattered clouds,-14.9261,-42.8192,
10,Coari,BR,73.98,overcast clouds,-4.0850,-63.1414,
23,Sao Filipe,CV,72.52,clear sky,14.8961,-24.4956,
28,Rio Grande,BR,70.95,light rain,-32.0350,-52.0986,
33,Senanga,ZM,65.39,light rain,-16.1167,23.2667,
37,Tocopilla,CL,68.99,few clouds,-22.0920,-70.1979,
44,Victoria,HK,61.18,broken clouds,22.2855,114.1577,
47,East London,ZA,69.28,light rain,-33.0153,27.9116,
52,Cabo San Lucas,MX,67.51,few clouds,22.8909,-109.9124,
53,Hilo,US,74.93,overcast clouds,19.7297,-155.0900,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,City Description,Lat,Lng,Hotel Name
2,Espinosa,BR,70.30,scattered clouds,-14.9261,-42.8192,Hotel Vilma
10,Coari,BR,73.98,overcast clouds,-4.0850,-63.1414,HOTEL ALEX FLAT
23,Sao Filipe,CV,72.52,clear sky,14.8961,-24.4956,Hotel Xaguate
28,Rio Grande,BR,70.95,light rain,-32.0350,-52.0986,Hotel Atlântico Rio Grande
33,Senanga,ZM,65.39,light rain,-16.1167,23.2667,Zango lodge
37,Tocopilla,CL,68.99,few clouds,-22.0920,-70.1979,Hotel Galvarino
44,Victoria,HK,61.18,broken clouds,22.2855,114.1577,Mini Hotel Central
47,East London,ZA,69.28,light rain,-33.0153,27.9116,Tu Casa
52,Cabo San Lucas,MX,67.51,few clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
53,Hilo,US,74.93,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>City Description</dt><dd>{City Description} at {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))